# **SCC0277:** Desafio de Regressão

Dikson F. Santos `<dikson@usp.br>`

`(github.com/diksown/barrels-of-insight)`


# Descrição

Utilizando a metodologia básica de um projeto para ciência de dados, implemente possíveis soluções para o seguinte estudo de caso (https://www.kaggle.com/datasets/podsyp/sales-in-craft-beer-bar). Desenvolva e implemente métodos que se propõem a estimar a variável de saída.

Em seu projeto deverá constar uma análise detalhada do dataset e descrição das transformações de dados realizadas com suas respectivas justificativas. Adicionalmente, crie um baseline com as técnicas contidas nesta aula. Como forma de superar este baseline, pesquise por métodos e técnicas mais complexas. Você deverá entregar um relatório com os seguintes itens:

Descrição do problema e análise dos dados
Descrição das técnicas utilizadas
Interpretação dos resultados obtidos
Conclusão
Apêndice (descrição das técnicas de Regressão apresentadas pelos colegas durante os seminários)
Adicionalmente, no relatório coloque a url de seu repositório on-line para consulta. Seu código deve estar comentado!!

Sistema de Avaliação:

- (1,0) Qualidade do relatório
- (1,0) Análise dos dados e descrição do problema
- (3,0) Implementação da solução
- (3,0) Interpretação dos resultados
- (2,0) Resumo das técnicas dos seminários de Regressão


# Introdução


O objetivo desse trabalho é desenvolver um modelo de machine learning para auxiliar na gestão do bar de cerveja Nelson Sauvin. Localizado no centro de Yekaterinburg, o bar se destaca por oferecer uma ampla variedade de cervejas interessantes de cervejarias new wave europeias, americanas e russas. Além disso, o estabelecimento conta com um estoque diversificado de garrafas e uma seleção dos melhores queijos regionais.


# Análise Exploratória de Dados e Pré-processamento


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

In [ ]:
sns.set_theme()

In [ ]:
product_range = pd.read_csv("./data/Product_range.csv")
transactions = pd.read_csv("./data/Transactions.csv")

In [ ]:
product_range.head(3)

In [ ]:
transactions.head(3)

In [ ]:
product_range.info()

In [ ]:
transactions.info()

In [ ]:
df = pd.merge(transactions, product_range, on="Product_code", how="inner")

In [ ]:
df.head(3)

In [ ]:
df.info()

In [ ]:
df = df.dropna()
df.info()

In [ ]:
def show_graph(col_name, description):
    category_count = df[col_name].value_counts().head(5)
    counts = category_count
    plt.figure(figsize=(8, 6))
    sns.barplot(x=counts.values, y=counts.index)
    plt.title(description)
    plt.xlabel("Frequência")
    plt.ylabel(None)
    plt.show()

In [ ]:
show_graph("Vendor_code", "Código do vendedor")

In [ ]:
show_graph("Name", "Produto")

In [ ]:
show_graph("Country_of_Origin", "País de origem")

In [ ]:
# Check the correlation matrix
numeric_columns = df.select_dtypes(include=[np.number])

correlation_matrix = numeric_columns.corr()

# Print the correlation matrix
# print(correlation_matrix)
# Create a heatmap plot
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", linewidths=0.5)
plt.title("Correlation Matrix")
plt.show()

# Treinamento do Modelo


In [ ]:
def evaluate_regressor(regressor):
    numeric_columns = df.select_dtypes(include=[np.number])
    to_predict = "Sale_amount"
    x, y = numeric_columns.drop(to_predict, axis=1), numeric_columns[to_predict]
    scores = cross_validate(
        regressor,
        x,
        y,
        cv=5,
        scoring=("neg_mean_squared_error", "r2"),
    )
    mse = scores["test_neg_mean_squared_error"].mean()
    r2 = scores["test_r2"].mean()
    print(f"Mean Squared Error: {mse:.2f}")
    print(f"R2 Score: {r2:.2f}")

In [ ]:
# Regressão Linear
evaluate_regressor(LinearRegression())